第1段階：元の３８個の特徴量から，論文に基づき選択した２５個の特徴量からLightGBMを用いて学習を行い，ハイパーパラメータチューニングを行ったモデルで，テストデータを「異常」と「正常」に分類する．
第二段階：選択した２５個の特徴量に加え，選択した2５個の特徴量を用いて作成したオートエンコーダのエンコーダ部分の特徴量5個を合わせた３０個の特徴量からLightGBMを用いて学習を行い，ハイパーパラメータチューニングを行ったモデルで，第1段階で「異常」と判断されたテストデータを4つの異常クラスに分類する．

In [4]:
# import modules
import pandas as pd
import numpy as np
import lightgbm as lgb


In [5]:
# model setting

# x_test_1st: 選択された25個の特徴量のみのデータセット
x_test_1st_path: str = "models/kdd99_features/x_test-drop_25_df.pkl"

# x_test_2nd: 選択された25個の特徴量とオートエンコーダのエンコーダ部分の特徴量5個を合わせた30個の特徴量のデータセット
x_test_2nd_path: str = "models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl"

# model_1st: 選択された25個の特徴量のみのデータセットを用いて学習を行い，ハイパーパラメータチューニングを行ったモデル
model_1st_path: str = "models/lightgbm/lgb_dropped_25_binary_tuned_booster.model"

# model_2nd: 選択された25個の特徴量とオートエンコーダのエンコーダ部分の特徴量5個を合わせた30個の特徴量のデータセットを用いて学習を行い，ハイパーパラメータチューニングを行ったモデル
model_2nd_path: str = "models/lightgbm/lgb_dropped+ae_30_mapped_anomaly_tuned_booster.model"

In [6]:
# load data

x_test_1st: pd.DataFrame = pd.read_pickle(x_test_1st_path)

x_test_2nd: pd.DataFrame = pd.read_pickle(x_test_2nd_path)

# y_test: 正常クラス(0)と4つの異常クラス(1,2,3,4)に分類されたデータセット
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")

# y_test_binary: 1つの正常クラス(0)と4つの異常クラス(1)に分類されたデータセット
y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)

# y_test_anomaly: 4つの異常クラスのみのデータセット
y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")

# モデルの読み込み
model_1st: lgb.Booster = lgb.Booster(model_file=model_1st_path)
model_2nd: lgb.Booster = lgb.Booster(model_file=model_2nd_path)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression


# functions
# 第一段階　2値分類
def classification_normal_and_anomaly(X: pd.DataFrame, model: LogisticRegression | lgb.Booster) -> pd.Series:
    y_pred = model.predict(X)
    y_pred = np.round(y_pred).astype("int64")
    if len(y_pred.shape) == 2:
        y_pred = np.argmax(y_pred, axis=1)
    y_pred = pd.Series(y_pred, index=X.index, )
    return y_pred


# 第二段階　異常データの分類
def classification_anomalies(X: pd.DataFrame, model: LogisticRegression | lgb.Booster) -> pd.Series:
    y_pred = model.predict(X)
    if len(y_pred.shape) == 2:
        y_pred = np.argmax(y_pred, axis=1)  # 一番大きい予測確率のクラスを予測クラスに
    y_pred = pd.Series(y_pred, index=X.index)
    return y_pred


# attack_class_labels -> key: class, value: list[label]
attack_class_labels = {
    'normal': ['normal'],
    'dos': ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop'],
    'u2r': ['buffer_overflow', 'loadmodule', 'perl', 'rootkit'],
    'r2l': ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster'],
    'probe': ['ipsweep', 'nmap', 'portsweep', 'satan']
}

# class -> int
correspondences = {
    'dos': 0,
    'normal': 1,
    'probe': 2,
    'r2l': 3,
    'u2r': 4
}

swapped_correspondences = {v: k for k, v in correspondences.items()}

correspondences_anomaly = {
    'dos': 0,
    'probe': 1,
    'r2l': 2,
    'u2r': 3
}

ignore_columns = ["hot", "num_compromised", "num_file_creations", "num_outbound_cmds", "is_host_login", "srv_count",
                  "srv_serror_rate", "same_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
                  "dst_host_diff_srv_rate"]


wrapper = {0: 0, 1: 2, 2: 3, 3: 4}

# attack_class_label -> key: label, value: class
attack_label_class = {}
for c, labels in attack_class_labels.items():
    for label in labels:
        attack_label_class[label] = c

def confusion_matrix_df(y_true, y_pred, labels=correspondences.keys()):

    return pd.DataFrame(confusion_matrix(y_true, y_pred),
                        index=["true_" + label for label in labels],
                        columns=labels)


In [11]:
 # 第1段階: 正常と異常の2値分類
y_pred_binary:pd.Series = classification_normal_and_anomaly(x_test_1st, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index

y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
cm_1st

,normal,anomaly
true_normal,32087,15
true_anomaly,24,130901


In [12]:
# 第二段階；異常の分類
x_anomalies: pd.DataFrame = x_test_2nd.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
# print(f"{y_pred_anomalies.value_counts()}")

y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())

swapped_correspondences = {v: k for k, v in correspondences.items()}
y_pred_value_counts = y_pred.value_counts()
y_pred_value_counts.index = y_pred_value_counts.index.map(lambda x: swapped_correspondences[x])
cm_2nd

,dos,normal,probe,r2l,u2r
true_dos,129177,4,0,0,0
true_normal,5,32087,7,1,2
true_probe,3,8,1344,0,0
true_r2l,1,5,1,361,4
true_u2r,0,7,0,0,10


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test.sort_index(), y_pred.sort_index()))
print(classification_report(y_test.sort_index(), y_pred.sort_index(), target_names=correspondences.keys()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    129181
           1       1.00      1.00      1.00     32102
           2       0.99      0.99      0.99      1355
           3       1.00      0.97      0.98       372
           4       0.62      0.59      0.61        17

    accuracy                           1.00    163027
   macro avg       0.92      0.91      0.92    163027
weighted avg       1.00      1.00      1.00    163027

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00    129181
      normal       1.00      1.00      1.00     32102
       probe       0.99      0.99      0.99      1355
         r2l       1.00      0.97      0.98       372
         u2r       0.62      0.59      0.61        17

    accuracy                           1.00    163027
   macro avg       0.92      0.91      0.92    163027
weighted avg       1.00      1.00      1.00    163027


In [10]:
y_pred_value_counts

dos       129186
normal     32111
probe       1352
r2l          362
u2r           16
dtype: int64